## ML2 Classification Project - Heart Disease Prediction

Author: Paula Gwanchele

Algorithms: Logistic Regression, Random Forest, Neural Net (MLP)

### 1) Load Modules + Data

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
    RocCurveDisplay
)

In [2]:
df = pd.read_csv("heart_disease_medical_data.csv")

print("Shape:", df.shape)
print("\nColumns:", df.columns.tolist())
print("\nHead:\n", df.head())

Shape: (303, 14)

Columns: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'heart_disease']

Head:
    age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   1       145   233    1        2      150      0      2.3      3   
1   67    1   4       160   286    0        2      108      1      1.5      2   
2   67    1   4       120   229    0        2      129      1      2.6      2   
3   37    1   3       130   250    0        0      187      0      3.5      3   
4   41    0   2       130   204    0        2      172      0      1.4      1   

    ca  thal  heart_disease  
0  0.0   6.0              0  
1  3.0   3.0              1  
2  2.0   7.0              1  
3  0.0   3.0              0  
4  0.0   3.0              0  


### 2) Basic Data Checks + EDA

Dataset description: 13 columns, 1 target variable
- `age` - age in years  
- `sex` - sex (1 = male; 0 = female)
- `cp`  -   chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
- `trestbps` - resting blood pressure (on admission to the hospital)
- `chol` -  serum cholestoral in mg/dl
- `fbs` -  fasting blood sugar > 120 mg/dl
- `restecg` - resting electrocardiographic results
- `thalach` - maximum heart rate achieved
- `exang` -  exercise induced angina
- `oldpeak` -  ST depression induced by exercise relative to rest
- `slope` - the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
- `ca` -  number of major vessels (0-3) colored by flourosopy
- `thal` - 3 = normal; 6 = fixed defect; 7 = reversable defect

In [3]:
print("\nMissing values per column:\n", df.isna().sum())

target_col = "heart_disease"   # in your CSV it is already binary (0/1)
assert target_col in df.columns, f"Target column '{target_col}' not found."

print("\nTarget distribution:\n", df[target_col].value_counts(dropna=False))
print("\nTarget distribution (%):\n", df[target_col].value_counts(normalize=True).round(3) * 100)


Missing values per column:
 age              0
sex              0
cp               0
trestbps         0
chol             0
fbs              0
restecg          0
thalach          0
exang            0
oldpeak          0
slope            0
ca               4
thal             2
heart_disease    0
dtype: int64

Target distribution:
 heart_disease
0    164
1    139
Name: count, dtype: int64

Target distribution (%):
 heart_disease
0    54.1
1    45.9
Name: proportion, dtype: float64


In [4]:
# Plot target distribution
plt.figure()
sns.countplot(x=target_col, data=df)
plt.title("Target Distribution (0 = No disease, 1 = Disease)")
plt.xlabel("heart_disease")
plt.ylabel("Count")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "target_distribution.png"), dpi=200)
plt.show()

: 

In [ ]:
# Correlation heatmap (numeric)
plt.figure(figsize=(10, 8))
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=False)
plt.title("Correlation Heatmap (numeric features)")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "correlation_heatmap.png"), dpi=200)
plt.show()

### 3) Train/Test Split

In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

print("\nTrain shape:", X_train.shape, "Test shape:", X_test.shape)

### 4) Preprocessing Pipeline

In [ ]:
# In this heart dataset, columns are numeric codes (still fine as numeric).
# We'll treat all as numeric, impute missing, then scale (needed for LR + MLP).
numeric_features = X.columns.tolist()

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features)
    ],
    remainder="drop"
)

### 5) Models + Hyperparameter Tuning

In [ ]:
log_reg = LogisticRegression(max_iter=2000, class_weight="balanced", random_state=42)

rf = RandomForestClassifier(
    random_state=42,
    class_weight="balanced"
)

mlp = MLPClassifier(
    max_iter=2000,
    random_state=42
)

# Wrap each in a pipeline (like the insurance example style)
pipe_lr = Pipeline(steps=[("prep", preprocessor), ("model", log_reg)])
pipe_rf = Pipeline(steps=[("prep", preprocessor), ("model", rf)])
pipe_mlp = Pipeline(steps=[("prep", preprocessor), ("model", mlp)])

In [ ]:
# Hyperparameter Tuning (GridSearchCV)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10],
    "model__penalty": ["l2"],
    "model__solver": ["lbfgs"]
}

param_grid_rf = {
    "model__n_estimators": [200, 500],
    "model__max_depth": [None, 5, 10],
    "model__min_samples_split": [2, 5],
    "model__min_samples_leaf": [1, 2]
}

param_grid_mlp = {
    "model__hidden_layer_sizes": [(32,), (64,), (64, 32)],
    "model__alpha": [0.0001, 0.001, 0.01],
    "model__learning_rate_init": [0.001, 0.01]
}

# We'll optimize for balanced accuracy (good habit if classes are uneven)
scoring = "balanced_accuracy"

grid_lr = GridSearchCV(pipe_lr, param_grid_lr, cv=cv, scoring=scoring, n_jobs=-1)
grid_rf = GridSearchCV(pipe_rf, param_grid_rf, cv=cv, scoring=scoring, n_jobs=-1)
grid_mlp = GridSearchCV(pipe_mlp, param_grid_mlp, cv=cv, scoring=scoring, n_jobs=-1)

print("\nTraining Logistic Regression (GridSearch)...")
grid_lr.fit(X_train, y_train)

print("Training Random Forest (GridSearch)...")
grid_rf.fit(X_train, y_train)

print("Training MLP Neural Network (GridSearch)...")
grid_mlp.fit(X_train, y_train)

best_models = {
    "Logistic Regression": grid_lr.best_estimator_,
    "Random Forest": grid_rf.best_estimator_,
    "Neural Net (MLP)": grid_mlp.best_estimator_
}

print("\nBest params:")
for name, grid in [("LR", grid_lr), ("RF", grid_rf), ("MLP", grid_mlp)]:
    print(f"{name}: {grid.best_params_} | best CV balanced_acc={grid.best_score_:.4f}")

### 6) Model Evaluation

In [ ]:
def evaluate_model(name, model, X_test, y_test):
    y_pred = model.predict(X_test)

    # probabilities for ROC-AUC (if available)
    y_proba = None
    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    bacc = balanced_accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    auc = None
    if y_proba is not None:
        auc = roc_auc_score(y_test, y_proba)

    cm = confusion_matrix(y_test, y_pred)

    print("\n" + "="*60)
    print(f"MODEL: {name}")
    print("="*60)
    print(f"Accuracy:          {acc:.4f}")
    print(f"Balanced Accuracy: {bacc:.4f}")
    print(f"Precision:         {prec:.4f}")
    print(f"Recall:            {rec:.4f}")
    print(f"F1-score:          {f1:.4f}")
    if auc is not None:
        print(f"ROC-AUC:           {auc:.4f}")

    print("\nConfusion Matrix:\n", cm)
    print("\nClassification Report:\n", classification_report(y_test, y_pred, digits=4))

In [ ]:
   # Save confusion matrix plot
    plt.figure()
    sns.heatmap(cm, annot=True, fmt="d")
    plt.title(f"Confusion Matrix - {name}")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, f"confusion_matrix_{name.replace(' ', '_')}.png"), dpi=200)
    plt.show()

In [ ]:

    # Save ROC curve if possible
    if y_proba is not None:
        plt.figure()
        RocCurveDisplay.from_predictions(y_test, y_proba)
        plt.title(f"ROC Curve - {name}")
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, f"roc_curve_{name.replace(' ', '_')}.png"), dpi=200)
        plt.show()

    return {
        "model": name,
        "accuracy": acc,
        "balanced_accuracy": bacc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": auc
    }

In [ ]:
# Save results
results = []
for name, model in best_models.items():
    res = evaluate_model(name, model, X_test, y_test)
    results.append(res)

results_df = pd.DataFrame(results).sort_values(by="balanced_accuracy", ascending=False)
print("\n=== Summary (sorted by balanced accuracy) ===\n", results_df)

results_path = os.path.join(OUTPUT_DIR, "model_results_summary.csv")
results_df.to_csv(results_path, index=False)
print("\nSaved results to:", results_path)

# -----------------------------
# 9) Save a cleaned version of the dataset (optional but good for submission)
# -----------------------------
clean_path = os.path.join(OUTPUT_DIR, "heart_disease_cleaned.csv")
df.to_csv(clean_path, index=False)
print("Saved cleaned dataset to:", clean_path)

ModuleNotFoundError: No module named 'seaborn'